[discussion 에 공유된 코드](https://www.kaggle.com/abhishek/two-longformers-are-better-than-1) 를 재생산하는 것을 목표로 한다.ds

[github 링크](https://github.com/abhishekkrthakur/long-text-token-classification)

#!# : todo list

# todo



In [92]:
import gc
import sys
import os
from tqdm import tqdm

import numpy as np
import pandas as pd
import random
import tez # https://www.notion.so/Tez-093f1f31cba646e3963108294563ddd1
from tez.callbacks import Callback
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoConfig, AutoModel, AutoTokenizer

from joblib import Parallel, delayed # https://www.notion.so/joblib-da8f5ee8dbd44da19b36da04bd657bb1

from einops import rearrange, repeat, reduce
from torch.utils.data import Dataset, DataLoader

gc.enable()
sys.path.append("../input/tez-lib/") #!#
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
target_id_map = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 2,
    "I-Position": 3,
    "B-Evidence": 4,
    "I-Evidence": 5,
    "B-Claim": 6,
    "I-Claim": 7,
    "B-Concluding Statement": 8,
    "I-Concluding Statement": 9,
    "B-Counterclaim": 10,
    "I-Counterclaim": 11,
    "B-Rebuttal": 12,
    "I-Rebuttal": 13,
    "O": 14,
    "PAD": -100,
}

In [81]:
# reproduce 마무리한 뒤에는 parser 함수로 바꿔주기
# https://github.com/abhishekkrthakur/long-text-token-classification/blob/8f636ea23b7e1842583581d9cbdbe9f0f54d3191/train.py#L30
class set_args: #1#
    seed: int = 42
    fold: int = 0
    kfold: int = 5
    model = 'allenai/longformer-base-4096' #!# 모델 이름
    lr: float = 1e-5
    output_path = os.path.join(os.getcwd(), 'model') # '../model'
    input_path = os.path.join(os.getcwd(), 'input') # '../input'
    max_len: int = 1024
    batch_size: int = 8
    valid_batch_size: int = 8
    epochs: int = 20
    accumulation_steps = 1 #!# 이게 뭐지?

In [82]:
def seed_everything(seed: int): #2#
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [83]:
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

def get_stratified_fold(df:pd.DataFrame, args): #3# ref : https://www.kaggle.com/abhishek/creating-folds-properly-hopefully-p/
    '''iterstrat 를 활용해서 multi-label 데이터에 대한 StratifiedKFold 를 진행한다.
        Parameters
            df : need to have 'id', 'discourse_type' columns.
            args : kfold 와 seed number 정보를 담고 있는 class.
        
        Returns
            df : 'kfold' column 이 추가된 df 를 반환한다.
    '''
    dfx = pd.get_dummies(df, columns=["discourse_type"]) # discourse_type 에 대한 one-hot vector 로 표현
    dfx = dfx.groupby(["id"], as_index=False).sum() # 각 text 에서의 discourse_type 갯수 계산하기
    
    cols = [c for c in dfx.columns if c.startswith("discourse_type_") or c == "id" and c != "discourse_type_num"] # discourse_type 열 추출
    dfx = dfx[cols]

    mskf = MultilabelStratifiedKFold(n_splits=args.kfold, shuffle=True, random_state=args.seed) # kfold 를 수행해주는 class 선언
    labels = [c for c in dfx.columns if c != "id"]
    dfx_labels = dfx[labels]
    dfx["kfold"] = -1 

    for fold, (trn_idx_array, val_idx_array) in enumerate(mskf.split(dfx, dfx_labels)): # mskf.split(dfx, dfx_labels) 는 (trn:array, val:array) index 를 내뱉어주는 generater.
        dfx.loc[val_idx_array, f"{args.kfold}fold"] = fold

    df = df.merge(dfx[["id", f"{args.kfold}fold"]], on="id", how="left") # on : id 를 기준으로 merge, how : matching 되는 id 가 없어도 남겨둔다 (left)
    df.to_csv(f"input/train_{args.kfold}folds.csv", index=False)
    return df

In [84]:
#4# preprocessing
def prepare_training_data(df: pd.DataFrame, tokenizer, args, num_jobs):
    '''_prepare_training_data_helper 를 병렬처리
        Parameters
            tokenizer : 전처리에 활용되는 tokenizer
            num_jobs : number of 병렬 처리 workers
        
        Returns
            training_samples(list) : 
    '''
    training_samples = []
    train_ids = df["id"].unique()

    train_ids_splits = np.array_split(train_ids, num_jobs)

    results = Parallel(n_jobs=num_jobs, backend="multiprocessing")(
        delayed(_prepare_training_data_helper)(df, tokenizer, args, idx) for idx in train_ids_splits
    )
    for result in results:
        training_samples.extend(result)

    return training_samples

import copy
def _prepare_training_data_helper(df, tokenizer, args, train_ids):
    '''
    '''
    training_samples = []
    for idx in train_ids:
        filename = os.path.join(args.input_path, "train", idx + ".txt")
        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=False,
            return_offsets_mapping=True,
        )
        input_ids = encoded_text["input_ids"]
        input_labels = copy.deepcopy(input_ids)
        offset_mapping = encoded_text["offset_mapping"]

        for k in range(len(input_labels)):
            input_labels[k] = "O"

        sample = {
            "id": idx,
            "input_ids": input_ids,
            "text": text,
            "offset_mapping": offset_mapping,
        }

        temp_df = df[df["id"] == idx]
        for _, row in temp_df.iterrows():
            text_labels = [0] * len(text)
            discourse_start = int(row["discourse_start"])
            discourse_end = int(row["discourse_end"])
            prediction_label = row["discourse_type"]
            text_labels[discourse_start:discourse_end] = [1] * (discourse_end - discourse_start)
            target_idx = []
            for map_idx, (offset1, offset2) in enumerate(encoded_text["offset_mapping"]):
                if sum(text_labels[offset1:offset2]) > 0:
                    if len(text[offset1:offset2].split()) > 0:
                        target_idx.append(map_idx)

            targets_start = target_idx[0]
            targets_end = target_idx[-1]
            pred_start = "B-" + prediction_label
            pred_end = "I-" + prediction_label
            input_labels[targets_start] = pred_start
            input_labels[targets_start + 1 : targets_end + 1] = [pred_end] * (targets_end - targets_start)

        sample["input_ids"] = input_ids
        sample["input_labels"] = input_labels
        training_samples.append(sample)
    return training_samples
        
import copy
def _prepare_training_data_helper_sentencebert(df, tokenizer, args, train_ids): #!#
    '''
    1. nltk 를 활용해서 text 를 sentence 단위로 분할 후, [SEP] token 을 집어넣는다.
    2. 이후 _prepare_training_data_helper 과 똑같은 방법으로 전처리한다.
    #!# nltk.sent_tokenize 의 경우 sentence 를 잘못 나누는 경우가 있다. sentence bert 를 사용시 주의할 것.
    #!# 이 방법으로 예측한 결과는 index 를 옮겨주는 후처리가 필요하다.
    '''
    training_samples = []
    for idx in train_ids:
        filename = os.path.join()

In [88]:
class FeedbackDataset(Dataset):
    def __init__(self, samples, max_len, tokenizer):
        super(FeedbackDataset, self).__init__() #!#
        self.samples = samples
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.length = len(samples)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        input_ids = self.samples[idx]["input_ids"]
        input_labels = self.samples[idx]["input_labels"]
        input_labels = [target_id_map[x] for x in input_labels]
        other_label_id = target_id_map["O"]
        padding_label_id = target_id_map["PAD"]
        # print(input_ids)
        # print(input_labels)

        # add start token id to the input_ids
        input_ids = [self.tokenizer.cls_token_id] + input_ids
        input_labels = [other_label_id] + input_labels

        if len(input_ids) > self.max_len - 1:
            input_ids = input_ids[: self.max_len - 1]
            input_labels = input_labels[: self.max_len - 1]

        # add end token id to the input_ids
        input_ids = input_ids + [self.tokenizer.sep_token_id]
        input_labels = input_labels + [other_label_id]

        attention_mask = [1] * len(input_ids)

        padding_length = self.max_len - len(input_ids)
        if padding_length > 0:
            if self.tokenizer.padding_side == "right":
                input_ids = input_ids + [self.tokenizer.pad_token_id] * padding_length
                input_labels = input_labels + [padding_label_id] * padding_length
                attention_mask = attention_mask + [0] * padding_length
            else:
                input_ids = [self.tokenizer.pad_token_id] * padding_length + input_ids
                input_labels = [padding_label_id] * padding_length + input_labels
                attention_mask = [0] * padding_length + attention_mask

        return {
            "ids": torch.tensor(input_ids, dtype=torch.long),
            "mask": torch.tensor(attention_mask, dtype=torch.long),
            "targets": torch.tensor(input_labels, dtype=torch.long),
        }
    
class FeedbackDatasetValid:
    def __init__(self, samples, max_len, tokenizer):
        self.samples = samples
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.length = len(samples)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        input_ids = self.samples[idx]["input_ids"]
        input_ids = [self.tokenizer.cls_token_id] + input_ids

        if len(input_ids) > self.max_len - 1:
            input_ids = input_ids[: self.max_len - 1]

        # add end token id to the input_ids
        input_ids = input_ids + [self.tokenizer.sep_token_id]
        attention_mask = [1] * len(input_ids)

        return {
            "ids": input_ids,
            "mask": attention_mask,
        }

In [108]:
#5# build model
from torch.optim import AdamW
class FeedbackModel(tez.Model):
    '''Kaggle baseline 모델
    특징
        - LM : hidden dropout prob = 0.1, layer_norm_eps = 1e-7
        - Head : dropout 을 0.1~0.5 로 설정하고 fc_layer 에 넣어준다. 이후 평균한 값을 예측에 활용한다.
        - loss : 평균하기 전 결과에 대해 training 을 진행한다. 일종의 ensemble 로 해석할 수 있을듯.
    '''
    def __init__(self, model_name, num_train_steps, learning_rate, num_labels, steps_per_epoch):
        super().__init__()
        self.learning_rate = learning_rate
        self.model_name = model_name
        self.num_train_steps = num_train_steps
        self.num_labels = num_labels
        self.steps_per_epoch = steps_per_epoch
        self.step_scheduler_after = "batch"

        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(model_name)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
                "num_labels": self.num_labels,
            }
        )
        
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)
        self.dropout4 = nn.Dropout(0.4)
        self.dropout5 = nn.Dropout(0.5)
        self.output = nn.Linear(config.hidden_size, self.num_labels)

    def fetch_optimizer(self):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.bias"]
        optimizer_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.01,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        opt = AdamW(optimizer_parameters, lr=self.learning_rate)
        return opt

    def fetch_scheduler(self):
        sch = get_cosine_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=int(0.1 * self.num_train_steps),
            num_training_steps=self.num_train_steps,
            num_cycles=1,
            last_epoch=-1,
        )
        return sch

    def loss(self, outputs, targets, attention_mask):
        loss_fct = nn.CrossEntropyLoss()

        active_loss = attention_mask.view(-1) == 1
        active_logits = outputs.view(-1, self.num_labels)
        true_labels = targets.view(-1)
        outputs = active_logits.argmax(dim=-1)
        idxs = np.where(active_loss.cpu().numpy() == 1)[0]
        active_logits = active_logits[idxs]
        true_labels = true_labels[idxs].to(torch.long)

        loss = loss_fct(active_logits, true_labels)
        return loss

    def monitor_metrics(self, outputs, targets, attention_mask):
        active_loss = (attention_mask.view(-1) == 1).cpu().numpy()
        active_logits = outputs.view(-1, self.num_labels)
        true_labels = targets.view(-1).cpu().numpy()
        outputs = active_logits.argmax(dim=-1).cpu().numpy()
        idxs = np.where(active_loss == 1)[0]
        f1_score = metrics.f1_score(true_labels[idxs], outputs[idxs], average="macro")
        return {"f1": f1_score}

    def forward(self, ids, mask, token_type_ids=None, targets=None):

        if token_type_ids:
            transformer_out = self.transformer(ids, mask, token_type_ids)
        else:
            transformer_out = self.transformer(ids, mask)
        sequence_output = transformer_out.last_hidden_state
        sequence_output = self.dropout(sequence_output)

        logits1 = self.output(self.dropout1(sequence_output))
        logits2 = self.output(self.dropout2(sequence_output))
        logits3 = self.output(self.dropout3(sequence_output))
        logits4 = self.output(self.dropout4(sequence_output))
        logits5 = self.output(self.dropout5(sequence_output))

        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        logits = torch.softmax(logits, dim=-1)
        loss = 0

        if targets is not None:
            loss1 = self.loss(logits1, targets, attention_mask=mask)
            loss2 = self.loss(logits2, targets, attention_mask=mask)
            loss3 = self.loss(logits3, targets, attention_mask=mask)
            loss4 = self.loss(logits4, targets, attention_mask=mask)
            loss5 = self.loss(logits5, targets, attention_mask=mask)
            loss = (loss1 + loss2 + loss3 + loss4 + loss5) / 5
            f1_1 = self.monitor_metrics(logits1, targets, attention_mask=mask)["f1"]
            f1_2 = self.monitor_metrics(logits2, targets, attention_mask=mask)["f1"]
            f1_3 = self.monitor_metrics(logits3, targets, attention_mask=mask)["f1"]
            f1_4 = self.monitor_metrics(logits4, targets, attention_mask=mask)["f1"]
            f1_5 = self.monitor_metrics(logits5, targets, attention_mask=mask)["f1"]
            f1 = (f1_1 + f1_2 + f1_3 + f1_4 + f1_5) / 5
            metric = {"f1": f1}
            return logits, loss, metric

        return logits, loss, {}

In [109]:
#6# 모델 후처리 및 tez 의 EarlyStopping module 정의
class EarlyStopping(Callback):
    def __init__(
        self,
        model_path,
        valid_df,
        valid_samples,
        batch_size,
        tokenizer,
        patience=5,
        mode="max",
        delta=0.001,
        save_weights_only=True,
    ):
        self.patience = patience # update 없이 흘러가는 epoch. 그래서 인내심인듯
        self.counter = 0 #!#
        self.mode = mode #!#
        self.best_score = None
        self.early_stop = False
        self.delta = delta # model update 의 최소 기준. delta 보다 작은 score 의 향상에 대해서는 model weight 저장이 이뤄지지 않는다.
        self.save_weights_only = save_weights_only
        self.model_path = model_path
        self.valid_samples = valid_samples
        self.batch_size = batch_size
        self.valid_df = valid_df
        self.tokenizer = tokenizer

        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def on_epoch_end(self, model):
        '''
        1. model.predict -> final_preds, final_class 를 self.valid_samples 에 저장
        2. model.predict 를 토대로 min_thresh, proba_thresh 를
        '''
        model.eval()
        valid_dataset = FeedbackDatasetValid(self.valid_samples, 4096, self.tokenizer)
        collate = Collate(self.tokenizer)

        #!# model.predict 는 tez 에서 제공하는 기능인듯.
        preds_iter = model.predict(
            valid_dataset,
            batch_size=self.batch_size,
            n_jobs=-1,
            collate_fn=collate,
        )
        
        final_preds = []
        final_scores = []
        for preds in preds_iter:
            pred_class = np.argmax(preds, axis=2)
            pred_scrs = np.max(preds, axis=2)
            for pred, pred_scr in zip(pred_class, pred_scrs):
                final_preds.append(pred.tolist())
                final_scores.append(pred_scr.tolist())

        for j in range(len(self.valid_samples)):
            tt = [id_target_map[p] for p in final_preds[j][1:]]
            tt_score = final_scores[j][1:]
            self.valid_samples[j]["preds"] = tt
            self.valid_samples[j]["pred_scores"] = tt_score

        submission = []
        min_thresh = {
            "Lead": 9,
            "Position": 5,
            "Evidence": 14,
            "Claim": 3,
            "Concluding Statement": 11,
            "Counterclaim": 6,
            "Rebuttal": 4,
        }
        proba_thresh = {
            "Lead": 0.7,
            "Position": 0.55,
            "Evidence": 0.65,
            "Claim": 0.55,
            "Concluding Statement": 0.7,
            "Counterclaim": 0.5,
            "Rebuttal": 0.55,
        }

        for _, sample in enumerate(self.valid_samples):
            preds = sample["preds"] # 각 token 들의 predict 결과 [id_target_map[p] for p in final_preds[j][1:]]
            offset_mapping = sample["offset_mapping"]
            sample_id = sample["id"]
            sample_text = sample["text"]
            sample_pred_scores = sample["pred_scores"]

            # pad preds to same length as offset_mapping
            if len(preds) < len(offset_mapping):
                preds = preds + ["O"] * (len(offset_mapping) - len(preds))
                sample_pred_scores = sample_pred_scores + [0] * (len(offset_mapping) - len(sample_pred_scores))

            #!# 
            idx = 0
            phrase_preds = []
            while idx < len(offset_mapping):
                start, _ = offset_mapping[idx]
                if preds[idx] != "O":
                    label = preds[idx][2:] #!# preds[idx][2:]
                else:
                    label = "O"
                phrase_scores = []
                phrase_scores.append(sample_pred_scores[idx])
                idx += 1
                while idx < len(offset_mapping):
                    if label == "O":
                        matching_label = "O"
                    else:
                        matching_label = f"I-{label}"
                    if preds[idx] == matching_label:
                        _, end = offset_mapping[idx]
                        phrase_scores.append(sample_pred_scores[idx])
                        idx += 1
                    else:
                        break # 해당 token 이 끝났기때문에 멈춘다.
                        
                if "end" in locals(): # _, end = offset_mapping[idx]
                    phrase = sample_text[start:end]
                    phrase_preds.append((phrase, start, end, label, phrase_scores))

            temp_df = []
            for phrase_idx, (phrase, start, end, label, phrase_scores) in enumerate(phrase_preds):
                word_start = len(sample_text[:start].split())
                word_end = word_start + len(sample_text[start:end].split())
                word_end = min(word_end, len(sample_text.split()))
                ps = " ".join([str(x) for x in range(word_start, word_end)])
                if label != "O":
                    if sum(phrase_scores) / len(phrase_scores) >= proba_thresh[label]:
                        temp_df.append((sample_id, label, ps))

            temp_df = pd.DataFrame(temp_df, columns=["id", "class", "predictionstring"])

            submission.append(temp_df)

        submission = pd.concat(submission).reset_index(drop=True)
        submission["len"] = submission.predictionstring.apply(lambda x: len(x.split()))

        # 각 token 이 점수를 넘었는지 boolean 으로 반환
        def threshold(df):
            df = df.copy()
            for key, value in min_thresh.items():
                index = df.loc[df["class"] == key].query(f"len<{value}").index
                df.drop(index, inplace=True)
            return df

        submission = threshold(submission)

        # drop len
        submission = submission.drop(columns=["len"])

        scr = score_feedback_comp(submission, self.valid_df, return_class_scores=True)
        print(scr)
        model.train()

        epoch_score = scr[0]
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print("EarlyStopping counter: {} out of {}".format(self.counter, self.patience))
            if self.counter >= self.patience:
                model.model_state = enums.ModelState.END
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print("Validation score improved ({} --> {}). Saving model!".format(self.val_score, epoch_score))
            model.save(self.model_path, weights_only=self.save_weights_only)
        self.val_score = epoch_score

class Collate:
    '''
    tokenizer 의 결과를 tensor 로 변환해준다.
    cf) padding 을 수작업으로 하는게 인상적이다..
    '''
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        output = dict()
        output["ids"] = [sample["ids"] for sample in batch]
        output["mask"] = [sample["mask"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["ids"]]
            output["mask"] = [s + (batch_max - len(s)) * [0] for s in output["mask"]]
        else:
            output["ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["ids"]]
            output["mask"] = [(batch_max - len(s)) * [0] + s for s in output["mask"]]

        # convert to tensors
        output["ids"] = torch.tensor(output["ids"], dtype=torch.long)
        output["mask"] = torch.tensor(output["mask"], dtype=torch.long)

        return output
        
def score_feedback_comp(pred_df, gt_df, return_class_scores=False):
    class_scores = {}
    pred_df = pred_df[["id", "class", "predictionstring"]].reset_index(drop=True).copy()
    for discourse_type, gt_subset in gt_df.groupby("discourse_type"):
        pred_subset = pred_df.loc[pred_df["class"] == discourse_type].reset_index(drop=True).copy()
        class_score = score_feedback_comp_micro(pred_subset, gt_subset)
        class_scores[discourse_type] = class_score
    f1 = np.mean([v for v in class_scores.values()])
    if return_class_scores:
        return f1, class_scores
    return f1

### test code

In [ ]:
NUM_JOBS = 12
args = set_args() #1# hyper parameter setting

seed_everything(args.seed) #2# fix seed for reproducability
os.makedirs(args.output_path, exist_ok = True)

#3# read kfold data
if os.path.isfile(os.path.join(args.input_path, f'/train_{args.kfold}folds.csv')):
    df = pd.read_csv(os.path.join(os.getcwd(), f'input/train_{args.kfold}folds.csv')) 
else:
    df = pd.read_csv(os.path.join(os.getcwd(), 'input/train.csv'))
    df = get_stratified_fold(df, args)
train_df = df[df[f"{args.kfold}fold"] != args.fold].reset_index(drop=True)
valid_df = df[df[f"{args.kfold}fold"] == args.fold].reset_index(drop=True)

#4# preprocess and wrap into dataset class
args.model = './model'

tokenizer = AutoTokenizer.from_pretrained(args.model) #!# 나중에 local 에서 불러오도록 코드 수정 
training_samples = prepare_training_data(train_df, tokenizer, args, num_jobs=NUM_JOBS)
valid_samples = prepare_training_data(valid_df, tokenizer, args, num_jobs=NUM_JOBS)

train_dataset = FeedbackDataset(training_samples, args.max_len, tokenizer)

 28%|██▊       | 293/1040 [00:05<00:17, 43.61it/s]

In [ ]:
#5# build model
num_train_steps = int(len(train_dataset) / args.batch_size / args.accumulation_steps * args.epochs)

model = FeedbackModel(
    model_name=args.model,
    num_train_steps=num_train_steps,
    learning_rate=args.lr,
    num_labels=len(target_id_map) - 1,
    steps_per_epoch=len(train_dataset) / args.batch_size,
)

#6# define early stopping module : tez module 활용을 위한 것.
es = EarlyStopping(
    model_path=os.path.join(args.output_path, f"model_{args.fold}.bin"),
    valid_df=valid_df,
    valid_samples=valid_samples,
    batch_size=args.valid_batch_size,
    patience=5,
    mode="max",
    delta=0.001,
    save_weights_only=True,
    tokenizer=tokenizer,
)

#7# train model
model.fit(
    train_dataset,
    train_bs=args.batch_size,
    device="cuda",
    epochs=args.epochs,
    callbacks=[es],
    fp16=True,
    accumulation_steps=args.accumulation_steps,
)

In [ ]:
if __name__ == '__main__':
    